In [ ]:
!pip -q install transformers accelerate safetensors bitsAndBytes


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

repo = "Luke-griggs/small-talk-1.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,  # or float16 if needed
)

tokenizer = AutoTokenizer.from_pretrained(repo)
model = AutoModelForCausalLM.from_pretrained(
    repo,
    torch_dtype=torch.float16,
    device_map="auto"
    )
model.eval()

In [ ]:
chat = [
    {"role": "system", 
     "content": """
You are a helpful, polite, and friendly assistant. Answer questions to the best of your ability.
If you don't know something, be honest and say so. Keep responses clear and concise."},

"""},
     {"role": "user", "content": "What's your favorite thing to do?"}
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )


full_text = tokenizer.decode(output[0], skip_special_tokens=True)
prompt_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)

print(full_text[len(prompt_text):])